# Data Analysis and Results Generation

This notebook performs exploratory data analysis and generates the final results for the dashboard. We'll focus on identifying patterns and trends in the music data.

In [12]:
# Import required libraries
import pandas as pd
import numpy as np
import glob
import json
import os
from datetime import datetime

# Configure directories
PROCESSED_DATA_DIR = "../data/processed"
DASHBOARD_DIR = "../dashboard"

# Ensure dashboard directory exists
os.makedirs(DASHBOARD_DIR, exist_ok=True)

def get_latest_file(pattern):
    """Get the most recent file matching the pattern."""
    files = glob.glob(pattern)
    if not files:
        raise FileNotFoundError(f"No files found matching {pattern}")
    return max(files, key=os.path.getctime)

# Load processed data
try:
    # Try parquet first, then CSV
    try:
        tracks_file = get_latest_file(f"{PROCESSED_DATA_DIR}/clean_tracks_*.parquet")
        tracks_df = pd.read_parquet(tracks_file)
        print(f"✅ Loaded parquet file: {tracks_file}")
    except FileNotFoundError:
        tracks_file = get_latest_file(f"{PROCESSED_DATA_DIR}/clean_tracks_*.csv")
        tracks_df = pd.read_csv(tracks_file)
        print(f"✅ Loaded CSV file: {tracks_file}")
    
    print(f"📊 Dataset shape: {tracks_df.shape}")
    print(f"Columns available: {list(tracks_df.columns)}")
    
except FileNotFoundError:
    print("❌ No processed data found. Please run the cleaning notebook first.")
    raise

✅ Loaded CSV file: ../data/processed\clean_tracks_20250919_120046.csv
📊 Dataset shape: (6928, 37)
Columns available: ['track_id', 'name', 'artist', 'album', 'popularity', 'duration_ms', 'playlist_id', 'duration_min', 'id', 'name_audio', 'artist_audio', 'album_audio', 'popularity_audio', 'duration_ms_audio', 'explicit', 'release_date', 'track_number', 'album_type', 'total_tracks', 'artist_id', 'album_id', 'duration_minutes', 'is_recent', 'is_single', 'preview_url', 'energy', 'valence', 'danceability', 'acousticness', 'instrumentalness', 'liveness', 'speechiness', 'tempo', 'loudness', 'key', 'mode', 'time_signature']


## Top Tracks Analysis

Identify the most popular tracks and their characteristics.

In [13]:
# Analyze top tracks with available data
print("🎵 Analyzing top tracks...")

# Get tracks with popularity data
if 'popularity' in tracks_df.columns:
    top_tracks = tracks_df.sort_values('popularity', ascending=False).head(100)
    print(f"✅ Found {len(top_tracks)} top tracks by popularity")
else:
    # Fallback: use all tracks if no popularity column
    top_tracks = tracks_df.head(100)
    print("⚠️ No popularity column found - using first 100 tracks")

# Build available columns for top tracks
available_cols = ['name', 'artist']
optional_cols = ['album', 'popularity', 'release_date', 'release_year', 
                'danceability', 'energy', 'valence', 'duration_min']

# Add ID column (could be 'id' or 'track_id')
if 'track_id' in tracks_df.columns:
    available_cols.insert(0, 'track_id')
elif 'id' in tracks_df.columns:
    available_cols.insert(0, 'id')

# Add optional columns that exist
for col in optional_cols:
    if col in tracks_df.columns:
        available_cols.append(col)

# Create top tracks info with available columns
top_tracks_info = top_tracks[available_cols].fillna('N/A').to_dict('records')
print(f"📋 Top tracks exported with columns: {available_cols}")

🎵 Analyzing top tracks...
✅ Found 100 top tracks by popularity
📋 Top tracks exported with columns: ['track_id', 'name', 'artist', 'album', 'popularity', 'release_date', 'danceability', 'energy', 'valence', 'duration_min']


## Feature Analysis

Analyze correlations between different audio features.

In [14]:
# Analyze feature correlations with available data
print("🔗 Analyzing feature correlations...")

# Define potential audio features
potential_features = ['danceability', 'energy', 'loudness', 'speechiness', 
                     'acousticness', 'instrumentalness', 'liveness', 'valence', 
                     'tempo', 'popularity']

# Get available features
available_features = [col for col in potential_features if col in tracks_df.columns]

if len(available_features) >= 2:
    # Calculate correlation matrix for available features
    correlation_matrix = tracks_df[available_features].corr()
    
    # Convert to dictionary for JSON
    feature_correlations = correlation_matrix.round(3).to_dict('index')
    
    print(f"✅ Correlation analysis completed for: {available_features}")
    print(f"📊 Feature matrix size: {correlation_matrix.shape}")
else:
    # Fallback: create empty correlation matrix
    feature_correlations = {}
    print("⚠️ Not enough numeric features for correlation analysis")
    print(f"Available features: {available_features}")

🔗 Analyzing feature correlations...
✅ Correlation analysis completed for: ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'popularity']
📊 Feature matrix size: (10, 10)


## Temporal Analysis

Analyze trends in musical features over time.

In [15]:
# Temporal analysis with available data
print("📅 Analyzing temporal trends...")

# Check if we have date information
date_col = None
if 'release_year' in tracks_df.columns:
    date_col = 'release_year'
elif 'release_date' in tracks_df.columns:
    # Extract year from release_date if needed
    tracks_df['release_year'] = pd.to_datetime(tracks_df['release_date'], errors='coerce').dt.year
    date_col = 'release_year'

if date_col is not None:
    # Define features for temporal analysis
    temporal_features = ['danceability', 'energy', 'valence', 'popularity']
    available_temporal_features = [col for col in temporal_features if col in tracks_df.columns]
    
    if available_temporal_features:
        # Calculate yearly averages
        yearly_stats = tracks_df.groupby(date_col)[available_temporal_features].mean()
        
        # Convert to dashboard format
        yearly_trends = yearly_stats.reset_index().round(3).to_dict('records')
        
        print(f"✅ Temporal analysis completed for: {available_temporal_features}")
        print(f"📈 Years covered: {tracks_df[date_col].min()} - {tracks_df[date_col].max()}")
    else:
        yearly_trends = []
        print("⚠️ No numeric features available for temporal analysis")
else:
    yearly_trends = []
    print("⚠️ No date information available for temporal analysis")

📅 Analyzing temporal trends...
✅ Temporal analysis completed for: ['danceability', 'energy', 'valence', 'popularity']
📈 Years covered: 1955.0 - 2025.0


## Generate data.json File

Generate the data.json file that will be used by the dashboard.

In [16]:
# Generate comprehensive dashboard data
print("💾 Generating dashboard data...")

# Ensure all required variables exist
if 'top_tracks_info' not in locals():
    top_tracks_info = []
if 'feature_correlations' not in locals():
    feature_correlations = {}
if 'yearly_trends' not in locals():
    yearly_trends = []
if 'available_features' not in locals():
    available_features = []

# Create comprehensive dashboard data
dashboard_data = {
    'metadata': {
        'total_tracks': len(tracks_df),
        'available_features': available_features,
        'columns': list(tracks_df.columns),
        'date_range': {
            'min_year': None,
            'max_year': None
        },
        'last_updated': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    'top_tracks': top_tracks_info,
    'feature_correlations': feature_correlations,
    'yearly_trends': yearly_trends,
    'summary_stats': {}
}

# Add date range if available
if 'release_year' in tracks_df.columns and tracks_df['release_year'].notna().any():
    dashboard_data['metadata']['date_range']['min_year'] = int(tracks_df['release_year'].min())
    dashboard_data['metadata']['date_range']['max_year'] = int(tracks_df['release_year'].max())

# Add summary statistics for numeric columns
numeric_cols = tracks_df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col not in ['track_id', 'id'] and not col.endswith('_audio'):  # Skip ID and duplicate columns
        try:
            col_data = tracks_df[col].dropna()  # Remove NaN values
            if len(col_data) > 0:
                dashboard_data['summary_stats'][col] = {
                    'mean': float(round(col_data.mean(), 3)),
                    'median': float(round(col_data.median(), 3)),
                    'std': float(round(col_data.std(), 3)),
                    'min': float(round(col_data.min(), 3)),
                    'max': float(round(col_data.max(), 3)),
                    'count': int(len(col_data))
                }
        except Exception as e:
            print(f"⚠️ Could not calculate stats for {col}: {e}")

# Save JSON file
try:
    output_path = f"{DASHBOARD_DIR}/data.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(dashboard_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Dashboard data saved: {output_path}")
    print(f"📊 Data includes:")
    print(f"   - {len(top_tracks_info)} top tracks")
    print(f"   - {len(feature_correlations)} feature correlations")
    print(f"   - {len(yearly_trends)} yearly trend points")
    print(f"   - {len(dashboard_data['summary_stats'])} feature statistics")
    print(f"   - Total dataset: {len(tracks_df)} tracks")
    
except Exception as e:
    print(f"❌ Error saving dashboard data: {e}")
    print(f"Error details: {str(e)}")
    
    # Save backup in processed data directory
    try:
        backup_path = f"{PROCESSED_DATA_DIR}/dashboard_data_backup.json"
        with open(backup_path, 'w', encoding='utf-8') as f:
            json.dump(dashboard_data, f, indent=2, ensure_ascii=False)
        print(f"💾 Backup saved: {backup_path}")
    except Exception as backup_error:
        print(f"❌ Could not save backup: {backup_error}")
        # Last resort: print data structure
        print("📋 Dashboard data structure:")
        for key, value in dashboard_data.items():
            if isinstance(value, (list, dict)):
                print(f"   {key}: {type(value).__name__} with {len(value)} items")
            else:
                print(f"   {key}: {type(value).__name__}")

print("\n🎉 Analysis complete!")

💾 Generating dashboard data...
✅ Dashboard data saved: ../dashboard/data.json
📊 Data includes:
   - 100 top tracks
   - 10 feature correlations
   - 66 yearly trend points
   - 19 feature statistics
   - Total dataset: 6928 tracks

🎉 Analysis complete!
